In [1]:
import matplotlib.pyplot as plt
import nibabel as nib
import pickle

import shutil
import glob
import json
import os
import numpy as np

from monai import transforms

# Prepare training and testing dataset

In [ ]:
imageTr_paths = sorted(glob.glob("G:/gitCode/MGH/data/ACDC/database/training/patient*/patient*_frame" + ('[0-9]' * 2) + ".nii.gz"))
labelTr_paths = sorted(glob.glob("G:/gitCode/MGH/data/ACDC/database/training/patient*/patient*_frame" + ('[0-9]' * 2) + "_gt.nii.gz"))
imageTs_paths = sorted(glob.glob("G:/gitCode/MGH/data/ACDC/database/testing/patient*/patient*_frame" + ('[0-9]' * 2) + ".nii.gz"))
labelTs_paths = sorted(glob.glob("G:/gitCode/MGH/data/ACDC/database/testing/patient*/patient*_frame" + ('[0-9]' * 2) + "_gt.nii.gz"))
infoTr_paths = sorted(glob.glob("G:/gitCode/MGH/data/ACDC/database/training/patient*/Info.cfg"))
infoTs_paths = sorted(glob.glob("G:/gitCode/MGH/data/ACDC/database/testing/patient*/Info.cfg"))

save_imgTr_dir = "G:/gitCode/MGH/ACDCTest/dataset/imagesTr"
save_lbTr_dir = "G:/gitCode/MGH/ACDCTest/dataset/labelsTr"
save_imgTs_dir = "G:/gitCode/MGH/ACDCTest/dataset/imagesTs"
save_lbTs_dir = "G:/gitCode/MGH/ACDCTest/dataset/labelsTs"
save_infoTr_dir = "G:/gitCode/MGH/ACDCTest/dataset/infosTr"
save_infoTs_dir = "G:/gitCode/MGH/ACDCTest/dataset/infosTs"

for path in imageTr_paths:
    save_path = os.path.join(save_imgTr_dir, os.path.split(path)[-1])
    shutil.copyfile(path, save_path)

for path in imageTs_paths:
    save_path = os.path.join(save_imgTs_dir, os.path.split(path)[-1])
    shutil.copyfile(path, save_path)

for path in labelTr_paths:
    save_path = os.path.join(save_lbTr_dir, os.path.split(path)[-1])
    shutil.copyfile(path, save_path)

for path in labelTs_paths:
    save_path = os.path.join(save_lbTs_dir, os.path.split(path)[-1])
    shutil.copyfile(path, save_path)

for path in infoTr_paths:
    
    save_path = os.path.join(save_infoTr_dir, '_'.join(os.path.normpath(path).split(os.path.sep)[-2:]))
    shutil.copyfile(path, save_path)

for path in infoTs_paths:
    save_path = os.path.join(save_infoTs_dir, '_'.join(os.path.normpath(path).split(os.path.sep)[-2:]))
    shutil.copyfile(path, save_path)

# Prepare Datsset Json

In [5]:
imageTr_paths = sorted(glob.glob("G:/gitCode/MGH/ACDCTest/dataset/imagesTr/patient*_frame" + ('[0-9]' * 2) + ".nii.gz"))
labelTr_paths = sorted(glob.glob("G:/gitCode/MGH/ACDCTest/dataset/labelsTr/patient*_frame" + ('[0-9]' * 2) + "_gt.nii.gz"))
imageTs_paths = sorted(glob.glob("G:/gitCode/MGH/ACDCTest/dataset/imagesTs/patient*_frame" + ('[0-9]' * 2) + ".nii.gz"))
labelTs_paths = sorted(glob.glob("G:/gitCode/MGH/ACDCTest/dataset/labelsTs/patient*_frame" + ('[0-9]' * 2) + "_gt.nii.gz"))
infoTr_paths = sorted(glob.glob("G:/gitCode/MGH/ACDCTest/dataset/infosTr/patient*_Info.cfg"))
infoTs_paths = sorted(glob.glob("G:/gitCode/MGH/ACDCTest/dataset/infosTs/patient*_Info.cfg"))

data_json = {"description": "acdc shihua",
             "name": 'acdc',
             "tensorImageSize": "3D",
             "modality": {"0": "MRI"},
             "labels": {
                 "0": "background",
                 "1": "RV",
                 "2": "myocardium",
                 "3": "LV"
             },
             "groups": {
                 "NOR": 0,
                 "MINF": 1,
                 "DCM": 2,
                 "HCM": 3,
                 "RV": 4
             }}

In [6]:
import random
np.random.seed(42)
selectedIDs = sorted(["patient" + str(id).zfill(3) for id in np.random.choice(range(1, 101), size=80, replace=False)])
selectedIDs

['patient001',
 'patient004',
 'patient005',
 'patient006',
 'patient007',
 'patient008',
 'patient009',
 'patient010',
 'patient011',
 'patient012',
 'patient013',
 'patient014',
 'patient016',
 'patient017',
 'patient018',
 'patient019',
 'patient020',
 'patient023',
 'patient025',
 'patient026',
 'patient027',
 'patient028',
 'patient029',
 'patient031',
 'patient032',
 'patient033',
 'patient034',
 'patient035',
 'patient036',
 'patient037',
 'patient039',
 'patient040',
 'patient041',
 'patient042',
 'patient043',
 'patient044',
 'patient045',
 'patient046',
 'patient047',
 'patient048',
 'patient049',
 'patient050',
 'patient051',
 'patient054',
 'patient055',
 'patient056',
 'patient057',
 'patient058',
 'patient059',
 'patient060',
 'patient062',
 'patient063',
 'patient065',
 'patient066',
 'patient067',
 'patient068',
 'patient069',
 'patient070',
 'patient071',
 'patient073',
 'patient074',
 'patient076',
 'patient077',
 'patient078',
 'patient079',
 'patient080',
 'patient0

In [ ]:
training = []
for img_path, lb_path in zip(imageTr_paths, labelTr_paths):
    patient_ID = os.path.split(img_path)[-1].split("_")[0]
    if patient_ID in selectedIDs:
        path_dic = {}
        patient_ID = os.path.split(img_path)[-1].split('_')[0]
        path_dic['image'] = '/'.join(os.path.normpath(img_path).split(os.path.sep)[-2:])
        path_dic['label'] = '/'.join(os.path.normpath(lb_path).split(os.path.sep)[-2:])
        path_dic['info'] = '/'.join(["infosTr", patient_ID+"_Info.cfg"])
        training.append(path_dic)

data_json['training'] = training

In [ ]:
validating = []
for img_path, lb_path in zip(imageTr_paths, labelTr_paths):
    patient_ID = os.path.split(img_path)[-1].split('_')[0]
    if patient_ID not in selectedIDs:
        path_dic = {}
        path_dic['image'] = '/'.join(os.path.normpath(img_path).split(os.path.sep)[-2:])
        path_dic['label'] = '/'.join(os.path.normpath(lb_path).split(os.path.sep)[-2:])
        path_dic['info'] = '/'.join(["infosTr", patient_ID+"_Info.cfg"])
        validating.append(path_dic)

data_json['validating'] = validating

In [ ]:
testing = []
for img_path, lb_path in zip(imageTs_paths, labelTs_paths):
    path_dic = {}
    patient_ID = os.path.split(img_path)[-1].split('_')[0]
    path_dic['image'] = '/'.join(os.path.normpath(img_path).split(os.path.sep)[-2:])
    path_dic['label'] = '/'.join(os.path.normpath(lb_path).split(os.path.sep)[-2:])
    path_dic['info'] = '/'.join(["infosTs", patient_ID+"_Info.cfg"])
    testing.append(path_dic)

data_json['testing'] = testing

In [ ]:

with open('G:/gitCode/MGH/ACDCTest/dataset/dataset.json', 'w', encoding='utf-8') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)

In [ ]:
with open('G:/gitCode/MGH/ACDCTest/dataset/dataset.pkl', 'wb') as file:
    pickle.dump(data_json, file)

# prepare ED 

In [7]:
all_train_IDs = sorted(["patient" + str(id).zfill(3) for id in range(1, 101)])
training = []
for patient_ID in all_train_IDs:
    if patient_ID in selectedIDs:
        path_dic = {}
        path_dic['info'] = '/'.join(["infosTr", patient_ID+"_Info.cfg"])

        with open(f"G:/gitCode/MGH/ACDCTest/dataset/infosTr/{patient_ID}_Info.cfg") as f:
                    lines = f.readlines()
                    for line in lines:
                        
                        items = line.strip().split(': ')
                        if items[0] == "ED":
                            path_dic['image'] = f"imagesTr/{patient_ID}_frame{str(items[1]).zfill(2)}.nii.gz"
                            path_dic['label'] = f"labelsTr/{patient_ID}_frame{str(items[1]).zfill(2)}_gt.nii.gz"
        training.append(path_dic)
    
data_json['training'] = training


validating = []
for patient_ID in all_train_IDs:
    if patient_ID not in selectedIDs:
        path_dic = {}
        path_dic['info'] = '/'.join(["infosTr", patient_ID+"_Info.cfg"])

        with open(f"G:/gitCode/MGH/ACDCTest/dataset/infosTr/{patient_ID}_Info.cfg") as f:
                    lines = f.readlines()
                    for line in lines:
                        
                        items = line.strip().split(': ')
                        if items[0] == "ED":
                            path_dic['image'] = f"imagesTr/{patient_ID}_frame{str(items[1]).zfill(2)}.nii.gz"
                            path_dic['label'] = f"labelsTr/{patient_ID}_frame{str(items[1]).zfill(2)}_gt.nii.gz"
        validating.append(path_dic)
    
data_json['validating'] = validating


all_test_IDs = sorted(["patient" + str(id).zfill(3) for id in range(101, 151)])
testing = []
for patient_ID in all_test_IDs:
    path_dic = {}
    path_dic['info'] = '/'.join(["infosTs", patient_ID+"_Info.cfg"])

    with open(f"G:/gitCode/MGH/ACDCTest/dataset/infosTs/{patient_ID}_Info.cfg") as f:
                lines = f.readlines()
                for line in lines:
                    
                    items = line.strip().split(': ')
                    if items[0] == "ED":
                        path_dic['image'] = f"imagesTs/{patient_ID}_frame{str(items[1]).zfill(2)}.nii.gz"
                        path_dic['label'] = f"labelsTs/{patient_ID}_frame{str(items[1]).zfill(2)}_gt.nii.gz"
    testing.append(path_dic)
    
data_json['testing'] = testing

with open('G:/gitCode/MGH/ACDCTest/dataset/dataset_ed.pkl', 'wb') as file:
    pickle.dump(data_json, file)


# prepare ES

In [8]:
all_train_IDs = sorted(["patient" + str(id).zfill(3) for id in range(1, 101)])
training = []
for patient_ID in all_train_IDs:
    if patient_ID in selectedIDs:
        path_dic = {}
        path_dic['info'] = '/'.join(["infosTr", patient_ID+"_Info.cfg"])

        with open(f"G:/gitCode/MGH/ACDCTest/dataset/infosTr/{patient_ID}_Info.cfg") as f:
                    lines = f.readlines()
                    for line in lines:
                        
                        items = line.strip().split(': ')
                        if items[0] == "ES":
                            path_dic['image'] = f"imagesTr/{patient_ID}_frame{str(items[1]).zfill(2)}.nii.gz"
                            path_dic['label'] = f"labelsTr/{patient_ID}_frame{str(items[1]).zfill(2)}_gt.nii.gz"
        training.append(path_dic)
    
data_json['training'] = training


validating = []
for patient_ID in all_train_IDs:
    if patient_ID not in selectedIDs:
        path_dic = {}
        path_dic['info'] = '/'.join(["infosTr", patient_ID+"_Info.cfg"])

        with open(f"G:/gitCode/MGH/ACDCTest/dataset/infosTr/{patient_ID}_Info.cfg") as f:
                    lines = f.readlines()
                    for line in lines:
                        
                        items = line.strip().split(': ')
                        if items[0] == "ES":
                            path_dic['image'] = f"imagesTr/{patient_ID}_frame{str(items[1]).zfill(2)}.nii.gz"
                            path_dic['label'] = f"labelsTr/{patient_ID}_frame{str(items[1]).zfill(2)}_gt.nii.gz"
        validating.append(path_dic)
    
data_json['validating'] = validating


all_test_IDs = sorted(["patient" + str(id).zfill(3) for id in range(101, 151)])
testing = []
for patient_ID in all_test_IDs:
    path_dic = {}
    path_dic['info'] = '/'.join(["infosTs", patient_ID+"_Info.cfg"])

    with open(f"G:/gitCode/MGH/ACDCTest/dataset/infosTs/{patient_ID}_Info.cfg") as f:
                lines = f.readlines()
                for line in lines:
                    
                    items = line.strip().split(': ')
                    if items[0] == "ES":
                        path_dic['image'] = f"imagesTs/{patient_ID}_frame{str(items[1]).zfill(2)}.nii.gz"
                        path_dic['label'] = f"labelsTs/{patient_ID}_frame{str(items[1]).zfill(2)}_gt.nii.gz"
    testing.append(path_dic)
    
data_json['testing'] = testing

with open('G:/gitCode/MGH/ACDCTest/dataset/dataset_es.pkl', 'wb') as file:
    pickle.dump(data_json, file)
